In [7]:
import pandas as pd
import numpy as np
import lucem_illud as li
import re

In [8]:
test_path = 'C:/Users/Zacha/repo/HomeworkNotebooks/data/incel_data/data/'

test_data = pd.read_json(test_path)

In [9]:
#make dataframe of messages
post_messages = test_data['messages']

In [34]:
# new dataframe with columns: username, user_title, post_time, joined, raw_text, clean_text, post_title
post_messages_df = pd.DataFrame(columns=['username', 'user_title', 'post_time', 'user_info', 'raw_text', 'clean_text', 'post_title'])

In [44]:
c = 0
message_df = pd.DataFrame(columns=['username', 'user_title', 'post_time', 'user_info', 'raw_text', 'clean_text', 'post_title'])
for row in test_data.iterrows():
    # print(row)
    for message in row[1]['messages']:
        # print(message)
        # message_df.loc[c] = [message['username'], message['user_title'], message['post_time'], message['joined'], message['raw_text'], message['clean_text'], row[1]['title']]
        message_df.loc[c, 'username'] = message['username']
        message_df.loc[c, 'user_title'] = message['user_title']
        message_df.loc[c, 'post_time'] = message['post_time']
        message_df.loc[c, 'user_info'] = message['joined']
        message_df.loc[c, 'raw_text'] = message['raw_text']
        message_df.loc[c, 'clean_text'] = message['clean_text']
        message_df.loc[c, 'post_title'] = row[1]['title']
        message_df.loc[c, 'post_author'] = row[1]['author']
        message_df.loc[c, 'post_label'] = row[1]['label']
        message_df.loc[c, 'post_time'] = row[1]['time']
        # message_df.loc[c, 'message']
        c += 1

In [68]:
def convertJSON(path):
    c = 0
    json_df = pd.read_json(path)
    df = pd.DataFrame(columns=['username', 'user_title', 'post_time', 'user_info', 'raw_text', 'clean_text', 'post_title'])
    for row in json_df.iterrows():
        k = 0
        for message in row[1]['messages']:
            df.loc[c, 'username'] = message['username']
            df.loc[c, 'user_title'] = message['user_title']
            df.loc[c, 'post_time'] = message['post_time']
            df.loc[c, 'user_info'] = message['joined']
            df.loc[c, 'raw_text'] = message['raw_text']
            df.loc[c, 'clean_text'] = message['clean_text']
            df.loc[c, 'post_title'] = row[1]['title']
            df.loc[c, 'post_author'] = row[1]['author']
            df.loc[c, 'post_label'] = row[1]['label']
            df.loc[c, 'post_time'] = row[1]['time']
            c += 1
            
            if k == 0:
                df.loc[c, 'first_message'] = True
            k += 1
    
    df['post_time'] = pd.to_datetime(df['post_time'], unit='s')
    
    df['user_info'] = df['user_info'].apply(incel_joined_read)
    df['user_info'] = df['user_info'].apply(list_to_dict)

    df['word_list'] = df['clean_text'].apply()
    return df   


In [45]:
def incel_joined_read(string):
    string = re.sub(r'\n', '|', string)
    string = string.split('|')
    string = [x for x in string if x != '']
    # string = [x for x in string if x != '' and x != 'Messages']
    return string

In [46]:
def list_to_dict(list):
    dict = {}
    for i in range(0, len(list), 2):
        try:
            dict[list[i]] = list[i+1]
        except:
            dict[list[i]] = ''
    
    return dict

In [47]:
message_df['user_info'] = message_df['user_info'].apply(incel_joined_read)
message_df['user_info'] = message_df['user_info'].apply(list_to_dict)

In [39]:
message_df.head()

,username,user_title,post_time,user_info,raw_text,clean_text,post_title,post_author,post_label
0,SergeantIncel,Admincel,1538414660,"{'Joined': 'Nov 7, 2017', 'Messages': '1,648'}",List of useful links:\n\nRules & FAQ\nIntroduc...,\nList of useful links:\n\nRules & FAQ\nIntrod...,Useful Links,SergeantIncel,NaN
1,MaxZM98,Socially constructed loser,1539783513,"{'Joined': 'May 2, 2018', 'Posts': '3,701', 'O...",I was looking at myself in the gym mirror and ...,\nI was looking at myself in the gym mirror an...,My arms are so small,MaxZM98,[SuicideFuel]
2,itsmemario,Banned,1539783513,"{'Joined': 'Sep 24, 2018', 'Posts': '581', 'On...","You, have a shitty training plan or shitty mea...","\nYou, have a shitty training plan or shitty m...",My arms are so small,MaxZM98,[SuicideFuel]
3,3uxp,Captain,1539783513,"{'Joined': 'Mar 9, 2018', 'Posts': '1,622', 'O...","I bet you're white, huge muscles and big arms ...","\nI bet you're white, huge muscles and big arm...",My arms are so small,MaxZM98,[SuicideFuel]
4,Hunter,Banned,1539783513,"{'Joined': 'Nov 28, 2017', 'Posts': '13,365', ...","@MaxZM98 \n\nNoFap, now!","\n@MaxZM98 \n\nNoFap, now!",My arms are so small,MaxZM98,[SuicideFuel]


In [16]:
message_df['lifetime message total'] = message_df['joined'].apply(lambda x: x[-1])

In [49]:
data_folder = 'C:/Users/Zacha/repo/HomeworkNotebooks/data/incel_data/data/'

df_1 = convertJSON(data_folder + '20181017133526_main.json')

In [53]:
df_1.to_csv('C:/Users/Zacha/repo/HomeworkNotebooks/data/incel_data/data/20181017133526_main.csv')

In [59]:
csv_folder = 'C:/Users/Zacha/repo/HomeworkNotebooks/data/incel_data/messages/'
df_csv = pd.read_csv(csv_folder + '20181017133526_main.csv', index_col=0)

In [63]:
def string_to_word_list(string):
    string = string.lower()
    string = re.sub(r'[^a-z0-9 ]', '', string)
    string = string.split(' ')
    string = [x for x in string if x != '']
    return string

In [64]:
df_1['word_list'] = df_1['clean_text'].apply(string_to_word_list)

In [65]:
df_1['token_list'] = 

,username,user_title,post_time,user_info,raw_text,clean_text,post_title,post_author,post_label,word_list
0,SergeantIncel,Admincel,2018-10-01 17:24:20,"{'Joined': 'Nov 7, 2017', 'Messages': '1,648'}",List of useful links:\n\nRules & FAQ\nIntroduc...,\nList of useful links:\n\nRules & FAQ\nIntrod...,Useful Links,SergeantIncel,NaN,"[list, of, useful, linksrules, faqintroduction..."
1,MaxZM98,Socially constructed loser,2018-10-17 13:38:33,"{'Joined': 'May 2, 2018', 'Posts': '3,701', 'O...",I was looking at myself in the gym mirror and ...,\nI was looking at myself in the gym mirror an...,My arms are so small,MaxZM98,[SuicideFuel],"[i, was, looking, at, myself, in, the, gym, mi..."
2,itsmemario,Banned,2018-10-17 13:38:33,"{'Joined': 'Sep 24, 2018', 'Posts': '581', 'On...","You, have a shitty training plan or shitty mea...","\nYou, have a shitty training plan or shitty m...",My arms are so small,MaxZM98,[SuicideFuel],"[you, have, a, shitty, training, plan, or, shi..."
3,3uxp,Captain,2018-10-17 13:38:33,"{'Joined': 'Mar 9, 2018', 'Posts': '1,622', 'O...","I bet you're white, huge muscles and big arms ...","\nI bet you're white, huge muscles and big arm...",My arms are so small,MaxZM98,[SuicideFuel],"[i, bet, youre, white, huge, muscles, and, big..."
4,Hunter,Banned,2018-10-17 13:38:33,"{'Joined': 'Nov 28, 2017', 'Posts': '13,365', ...","@MaxZM98 \n\nNoFap, now!","\n@MaxZM98 \n\nNoFap, now!",My arms are so small,MaxZM98,[SuicideFuel],"[maxzm98, nofap, now]"


In [66]:
df_sample = df_1.sample(n=20)

In [67]:
df_sample.head()

,username,user_title,post_time,user_info,raw_text,clean_text,post_title,post_author,post_label,word_list
10632,SpringHeeledJack,"18, EthnicCel, HairCel, NoseCel, LocationCel",2018-11-16 17:13:41,"{'Joined': 'Jul 29, 2018', 'Messages': '1,576'}",GermanCel27 said:\n\n\n\ncome to germany. and ...,\tGermanCel27 said: come to germany. and you w...,Seeing teenagers kissing on the street,Landru,[SuicideFuel],"[germancel27, said, come, to, germany, and, yo..."
15654,VirtueSignaller,Officer,2018-11-16 18:00:44,"{'Joined': 'Jul 24, 2018', 'Messages': '555'}",RageAgainstTDL said:\n\n\n\nNo they never will...,\tRageAgainstTDL said: No they never will. Loo...,What would be the best solution to inceldom in...,Mainländer,[Experiment],"[rageagainsttdl, said, no, they, never, will, ..."
2041,SniiperCel,(CTYR) - Cope Til' You Rope. Top Frenchcel.,2018-05-23 11:25:46,"{'Joined': 'Aug 3, 2018', 'Posts': '163', 'Onl...","Izayacel said:\n\n\n\nkeep tryin, you need fuc...","\tIzayacel said: keep tryin, you need fuckign ...",I will approach 1000 girls and find a GF,uninstall,[LifeFuel],"[izayacel, said, keep, tryin, you, need, fucki..."
15827,fOreVER,امر بالمعروف والنهي عن المنكر,2018-11-16 22:11:51,"{'Joined': 'Jul 8, 2018', 'Posts': '4,121', 'O...",SecondComing said:\n\n\n\nIgnore the sandnigge...,\tSecondComing said: Ignore the sandnigger. He...,italian male model asks 100 italian sluts for sex,Dota,None,"[secondcoming, said, ignore, the, sandnigger, ..."
12028,Zielony4,Mopping subhuman tears as Janitorcel.,2018-11-16 17:13:41,"{'Joined': 'Nov 8, 2017', 'Messages': '4,128'}",This always ruins my whole day and the followi...,\nThis always ruins my whole day and the follo...,Seeing teenagers kissing on the street,Landru,[SuicideFuel],"[this, always, ruins, my, whole, day, and, the..."
